In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display
import requests as re
from io import BytesIO
from os import path
from unidecode import unidecode


from core.downloads.geosampa import get_capabilities, get_features
from core.utils.cleaning import clean_subprefeitura

# GT Saúde

O Grupo de Trabalho de Saúde definiu os indicadores sem a utilização dos formulários, em reuniões posteriores. Por isso, os indicadores serão descritos diretamente neste notebook para documentação.

A base inicial para os indicadores é o capítulo 7, Eficiência nas macrorregiões de saúde no Sistema Único de Saúde: uma abordagem comparativa – 2008-2017, do livro [SUS: avaliação da eficiência do gasto público em saúde](https://repositorio.ipea.gov.br/handle/11058/12029). Este capítulo busca associar os custos e infraestrutura relacionada ao SUS e o impacto na saúde da população de cada macrorregião, utilizando as seguintes variáveis:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Despesas per capita em atenção básica e vigilância;
- Despesas per capita em atenção especializada e Suporte profilático terapêutico;
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Equipamentos respiradores e ventiladores SUS/hab. (x100.000);
- População (em mil hab.);
- Área (km2);
- Densidade demográfica (hab./km²);
- Idosos %;
- PIB per capita;
- Usuários de planos de saúde %.


A partir dessas referência, o GT avaliou a disponibilidade de dados semelhantes a nível intramunicipal e complementou com informações de produção dos serviços de saúde, chegando na seguinte lista de variáveis, sempre que possível agregando por Subprefeitura:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Leitos de UTI/hab. (x100.000);
- Despesas liquidadas em atenção básica e vigilância;
- Despesas liquidadas em atenção especializada e Suporte profilático terapêutico;
- Consultas na Atenção Básica;
- Consultas na Atenção Especializada;
- Consultas na Urgência/Emergência;
- Número de Unidades Básicas de Saúde;
- Número de Ambulatórios especializados;
- Número de Unidades de Saúde Mental;
- Número de Unidades de Urgência e Emergência;
- Número de Hospitais;
- Número de Unidades DST/AIDS;
- Número de demais unidades;

A partir dessa definição, vamos obter as variáveis.

# Extração das variáveis e transformação inicial

## Número de unidades

Estabelecimentos de saúde municipais, estaduais, federais e privados do município estão disponíveis no GeoSampa, em diferentes camadas. Primeiro, vamos avaliar quais são essas camadas e, depois, baixá-las e agrupá-las da maneira decidida pelo GT.

In [2]:
equipamento_saude_typenames = get_capabilities('equipamento_saude')
equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_abrangencia_ubs',
  'title': 'Abrangência das Unidades Básicas de Saúde',
  'abstract': 'Áreas de abrangência das UBS.'},
 {'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_coordenadoria_regional',
  'title': 'Coordenadoria Regional de Saúde',
  'abstract': 'Limites administrativos da Coordenadoria Regional de Saúde.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que real

Primeiro, removemos todas as camadas que não representam os pontos das unidades de saúde. Além disso, a camada de vigilância não retorna nenhuma unidade, então vamos removê-la também.

In [3]:
del equipamento_saude_typenames[0]
del equipamento_saude_typenames[1]
del equipamento_saude_typenames[4]
del equipamento_saude_typenames[8]
del equipamento_saude_typenames[7]

equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que realizam procedimentos de média e alta complexidade.'},
 {'name': 'geoportal:equipamento_saude_outros',
  'title': 'Saúde - Outros',
  'abstract': 'Estabelecimentos e serviços especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo.'},
 {'name': 'geoportal:equipamento_saude_saude_mental

In [4]:
gdfs_equip = []

for t in equipamento_saude_typenames:
    gdfs_equip.append(get_features(t.get('name')))

In [5]:
for gdf in gdfs_equip:
    display(gdf.crs)
    display(gdf.head(2))

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,1,CAPAO REDONDO - AMA ESPECIALIDADES,"AV COMEN SANT'ANNA, 774",JD. BOA ESPERANCA,05866000\n,None,58729902,None,POINT (319108.42 7380764.47)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,2,PQ PERUCHE-AMA ESPECIALIDADES,"R JOSE RANGEL DE CAMARGO, 500",PQ PERUCHE,02538010\n,None,38589865,None,POINT (331055.43 7400384.23)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_hospital.120794,120794,6,SAUDE,0,SEM REDE,Sem descrição,30,Hospital,Realiza atendimento de média e alta complexidade.,...,Administração privada (inclui estabelecimentos...,281,SANTA MARIA CRUZ AZUL - HOSP,"AV LINS DE VASCONCELOS, 356",CAMBUCI,01538000\n,None,33484000,None,POINT (334481.53 7392822.24)
1,equipamento_saude_hospital.120795,120795,6,SAUDE,0,SEM REDE,Sem descrição,30,Hospital,Realiza atendimento de média e alta complexidade.,...,Administração privada (inclui estabelecimentos...,282,CASA DA CRIANCA BETINHO LAR ESPIRITA P/ EXCEPC...,"R VACANGA, 300",V CARRAO,03433020\n,None,67817366,None,POINT (343610.97 7393605.47)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

...

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ubs_posto_centro.120271,120271,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",...,Administração municipal,16,UBS JARDIM CLIPER,"R BELIZARIO FERREIRA LIMA,11",JD BELA VISTA,04829020\n,None,59285703,None,POINT (326077.574 7375311.115)
1,equipamento_saude_ubs_posto_centro.120272,120272,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",...,Administração municipal,206,JD SAO FRANCISCO II - AMA/UBS INTEGRADA,"R BANDEIRA DE ARACAMBI, 704",JD SAO FRANCISCO,08310010\n,None,27516712,None,POINT (351299.99 7385828.42)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_unidades_dst-aids.121019,121019,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,...,Administração municipal,219,PQ IPE- CTA DST/AIDS,"R FRANCISCO ANTUNES MEIRA, 255",PQ IPE,05742060\n,None,58428962,None,POINT (320361.91 7384500.83)
1,equipamento_saude_unidades_dst-aids.121020,121020,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,...,Administração municipal,220,CIDADE TIRADENTES-CTA DST/AIDS,"R LUIS BORDESE, 96",CIDADE TIRADENTES,08471780\n,None,22827055,None,POINT (356673.7 7389004.94)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_urgencia_emergencia.120738,120738,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,41,COMPLEXO PRATES - AMA,"R PRATES, 1101",BOM RETIRO,01009000\n,None,32298192,None,POINT (333142.32 7397479.26)
1,equipamento_saude_urgencia_emergencia.120739,120739,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,42,JD ELISA MARIA - AMA,"R ROMULO NALDI, 57",JD ELISA MARIA,02873250\n,None,39810967,None,POINT (327799.8 7404943.83)


In [6]:
gdf_equip = pd.concat(gdfs_equip)
gdf_equip = gpd.GeoDataFrame(gdf_equip)
gdf_equip = gdf_equip.set_geometry(gdfs_equip[0].geometry.name)
gdf_equip = gdf_equip.set_crs(gdfs_equip[0].crs)
gdf_equip

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,1,CAPAO REDONDO - AMA ESPECIALIDADES,"AV COMEN SANT'ANNA, 774",JD. BOA ESPERANCA,05866000\n,None,58729902,None,POINT (319108.42 7380764.47)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,2,PQ PERUCHE-AMA ESPECIALIDADES,"R JOSE RANGEL DE CAMARGO, 500",PQ PERUCHE,02538010\n,None,38589865,None,POINT (331055.43 7400384.23)
2,equipamento_saude_ambulatorios_especializados....,120003,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,3,V CONSTANCIA -AMA ESPECIALIDADES,"R HERMENEGILDO MARTINI, SN",V CONSTANCIA,04438280\n,None,55626944,None,POINT (330196.17 7380955.02)
3,equipamento_saude_ambulatorios_especializados....,120004,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,4,JD ICARAI I (QUINTANA) -AMA ESPECIALIDADES,"R S ROQUE DO PARAGUACU, 190",V QUINTANA,04837150\n,None,59280272,None,POINT (326183.88 7373067.87)
4,equipamento_saude_ambulatorios_especializados....,120005,6,SAUDE,0,SEM REDE,Sem descrição,49,AMA Especialidades,Atende aos pacientes encaminhados pelas demais...,...,Administração municipal,5,ITAQUERA-AMA ESPECIALIDADES,"R AMERICO SALVADOR NOVELLI, 265",ITAQUERA,08210090\n,None,20712422,None,POINT (351434.05 7396009.19)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,equipamento_saude_urgencia_emergencia.120789,120789,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1169,UPA MOOCA III DOM PAULO EVARISTO ARNS,R DR FOMM 261,BELENZINHO,03163030\n,None,None,None,POINT (337224.82 7395152.06)
52,equipamento_saude_urgencia_emergencia.120790,120790,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1180,UPA CIDADE TIRADENTES,R IGARAPE DA DIANA 1,CONJ HAB INACIO MONTEIRO,08472170\n,None,None,None,POINT (356810.52 7392545.06)
53,equipamento_saude_urgencia_emergencia.120791,120791,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1181,UPA CITY JARAGUA,ESTRADA DAS TAIPAS 1648,JARAGUA,02991000\n,None,None,None,POINT (322821.39 7406568.93)
54,equipamento_saude_urgencia_emergencia.120792,120792,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1182,UPA JABAQUARA,R CRUZ DAS ALMAS 290,VILA CAMPESTRE,04330060\n,None,2476-7356,None,POINT (332239.79 7383268.35)


## Consultas médicas

Os dados de consultas médicas serão obtidos do boletim "Saúde em dados", disponibilizado anualmente pela CEINFO, da Secretaria Municipal de Saúde. Existe uma versão do boletim em excel, regionalizado por subprefeituras. Primeiro, vamos confirmar as planilhas disponíveis no arquivo.

In [7]:
url = 'https://capital.sp.gov.br/documents/d/saude/tabelas_ceinfo_dados_sub_2024_v3_rev09012025'

response = re.get(url)
response.raise_for_status()
xlsx_file = BytesIO(response.content)
excel = pd.ExcelFile(xlsx_file)

excel.sheet_names

['População',
 'Nascidos Vivos',
 'Grupos de Robson',
 'Mortalidade',
 'Neoplasia',
 'Doenças Notificação Compulsória',
 'Agravos',
 'COVID-19',
 'Estado Nutricional',
 'Cobertura Assist SIGA-Saúde',
 'Consultas Medicas_Odontológicas',
 'Consultas Médicas Especializada',
 'Cons Prof Nivel Superior SMS',
 'Cons Prof Nivel Superior SES',
 'Assistência Hospitalar',
 'Apoio Diagnóstico',
 'Rede Física',
 'Leitos']

A tabela desejada é a `Consultas Medicas_Odontológicas`, então vamos abri-la como um dataframe.

In [8]:
sheet_name = 'Consultas Medicas_Odontológicas'
header = [0, 1]
skiprows = 5
df_consultas = pd.read_excel(url,
                   sheet_name=sheet_name,
                   skiprows=skiprows,
                   header=header,
                   thousands='.')
df_consultas

Unnamed: 0_level_0                                      Subprefeitura  \
   Unnamed: 0_level_1                                 Unnamed: 1_level_1   
0                 NaN                          Aricanduva/Formosa/Carrão   
1                 NaN                                            Butantã   
2                 NaN                                        Campo Limpo   
3                 NaN                                  Capela do Socorro   
4                 NaN                            Casa Verde/Cachoeirinha   
5                 NaN                                      Cidade Ademar   
6                 NaN                                  Cidade Tiradentes   
7                 NaN                                 Ermelino Matarazzo   
8                 NaN                              Freguesia/Brasilândia   
9                 NaN                                         Guaianases   
10                NaN                                           Ipiranga   
11                NaN                                     Itaim Paulista   
12                NaN                                           Itaquera   
13                NaN                                          Jabaquara   
14                NaN                                    Jaçanã/Tremembé   
15                NaN                                               Lapa   
16                NaN                                        M'Boi Mirim   
17                NaN                                              Mooca   
18                NaN                                        Parelheiros   
19                NaN                                              Penha   
20                NaN                                              Perus   
21                NaN                                          Pinheiros   
22                NaN                                   Pirituba/Jaraguá   
23                NaN                                   Santana/Tucuruvi   
24                NaN                                        Santo Amaro   
25                NaN                                         São Mateus   
26                NaN                                São Miguel Paulista   
27                NaN                                          Sapopemba   
28                NaN                                                 Sé   
29                NaN                          Vila Maria/Vila Guilherme   
30                NaN                                       Vila Mariana   
31                NaN                                      Vila Prudente   
32                NaN                             Município de São Paulo   
33                NaN                                                NaN   
34                NaN  Fonte: Sistema de Informação Ambulatorial - SI...   
35                NaN                                                NaN   
36                NaN  Nota: (1) A Rede SUS é composta por estabeleci...   
37                NaN                                                NaN   
38                NaN                                                NaN   
39                NaN                                                NaN   
40                NaN                                                NaN   
41                NaN                                                NaN   

   Consulta Médica\nna Atenção Básica(2)                            \
                       Não Urgência\nSMS Urgência\nSMS       Total   
0                               188894.0       93846.0    282740.0   
1                               443917.0      212283.0    656200.0   
2                               837251.0      652006.0   1489257.0   
3                               554063.0      283894.0    837957.0   
4                               327363.0      258095.0    585458.0   
5                               645314.0      188878.0    834192.0   
6                               325739.0       34662.0    360401.0   
7                           

Como a tabela original vem em um formato amigável para humanos, precisamos ajustá-la para melhorar a sua utilização computacional. A primeira coisa é filtrar linhas e colunas carregadas por engano, por conter informações de fonte ou caracteres não imprimíveis.

In [9]:
df_consultas = df_consultas.iloc[:32, 1:]

df_consultas

Subprefeitura Consulta Médica\nna Atenção Básica(2)  \
           Unnamed: 1_level_1                     Não Urgência\nSMS   
0   Aricanduva/Formosa/Carrão                              188894.0   
1                     Butantã                              443917.0   
2                 Campo Limpo                              837251.0   
3           Capela do Socorro                              554063.0   
4     Casa Verde/Cachoeirinha                              327363.0   
5               Cidade Ademar                              645314.0   
6           Cidade Tiradentes                              325739.0   
7          Ermelino Matarazzo                              258367.0   
8       Freguesia/Brasilândia                              476514.0   
9                  Guaianases                              358419.0   
10                   Ipiranga                              436257.0   
11             Itaim Paulista                              506647.0   
12                   Itaquera                              616893.0   
13                  Jabaquara                              202175.0   
14            Jaçanã/Tremembé                              333609.0   
15                       Lapa                              185920.0   
16                M'Boi Mirim                              927985.0   
17                      Mooca                              237620.0   
18                Parelheiros                              165641.0   
19                      Penha                              395582.0   
20                      Perus                              174019.0   
21                  Pinheiros                               98130.0   
22           Pirituba/Jaraguá                              499630.0   
23           Santana/Tucuruvi                              132926.0   
24                Santo Amaro                              115999.0   
25                 São Mateus                              578502.0   
26        São Miguel Paulista                              525496.0   
27                  Sapopemba                              388165.0   
28                         Sé                              308922.0   
29  Vila Maria/Vila Guilherme                              312691.0   
30               Vila Mariana                               90845.0   
31              Vila Prudente                              257419.0   

                            Consulta Médica\nna Atenção Especializada  \
   Urgência\nSMS      Total                              Gestão \nSMS   
0        93846.0   282740.0                                     56651   
1       212283.0   656200.0                                     75444   
2       652006.0  1489257.0                                    217035   
3       283894.0   837957.0                                    184580   
4       258095.0   585458.0                                    155265   
5       188878.0   834192.0                                    130523   
6        34662.0   360401.0                                     37112   
7       162909.0   421276.0                                     81975   
8       120546.0   597060.0                                    167701   
9        50300.0   408719.0                                     42217   
10      253211.0   689468.0                                    235574   
11        5370.0   512017.0                                     60996   
12      166986.0   783879.0                                    166236   
13      106137.0   308312.0                                     83267   
14       68684.0   402293.0                                     38275   
15      191187.0   377107.0                                    132722   
16      517440.0  1445425.0                                    248403   
17      118979.0   356599.0                                    487398   
18       67455.0   233096.0                                     23280   
19      564046.0   959628.0                                    13358

Depois, precisamos remover as colunas de totais, que resultariam em duplicação dos valores caso fossem mantidas.

In [10]:
cols_total_level0 = df_consultas.columns.get_level_values(0).str.contains('total', case=False)
cols_total_level1 = df_consultas.columns.get_level_values(1).str.contains('total', case=False)

df_consultas = df_consultas.loc[:, ~cols_total_level0 & ~cols_total_level1]

df_consultas

Subprefeitura Consulta Médica\nna Atenção Básica(2)  \
           Unnamed: 1_level_1                     Não Urgência\nSMS   
0   Aricanduva/Formosa/Carrão                              188894.0   
1                     Butantã                              443917.0   
2                 Campo Limpo                              837251.0   
3           Capela do Socorro                              554063.0   
4     Casa Verde/Cachoeirinha                              327363.0   
5               Cidade Ademar                              645314.0   
6           Cidade Tiradentes                              325739.0   
7          Ermelino Matarazzo                              258367.0   
8       Freguesia/Brasilândia                              476514.0   
9                  Guaianases                              358419.0   
10                   Ipiranga                              436257.0   
11             Itaim Paulista                              506647.0   
12                   Itaquera                              616893.0   
13                  Jabaquara                              202175.0   
14            Jaçanã/Tremembé                              333609.0   
15                       Lapa                              185920.0   
16                M'Boi Mirim                              927985.0   
17                      Mooca                              237620.0   
18                Parelheiros                              165641.0   
19                      Penha                              395582.0   
20                      Perus                              174019.0   
21                  Pinheiros                               98130.0   
22           Pirituba/Jaraguá                              499630.0   
23           Santana/Tucuruvi                              132926.0   
24                Santo Amaro                              115999.0   
25                 São Mateus                              578502.0   
26        São Miguel Paulista                              525496.0   
27                  Sapopemba                              388165.0   
28                         Sé                              308922.0   
29  Vila Maria/Vila Guilherme                              312691.0   
30               Vila Mariana                               90845.0   
31              Vila Prudente                              257419.0   

                 Consulta Médica\nna Atenção Especializada               \
   Urgência\nSMS                              Gestão \nSMS Gestão \nSES   
0        93846.0                                     56651            -   
1       212283.0                                     75444        98066   
2       652006.0                                    217035        10527   
3       283894.0                                    184580       108804   
4       258095.0                                    155265        61589   
5       188878.0                                    130523            -   
6        34662.0                                     37112            -   
7       162909.0                                     81975            -   
8       120546.0                                    167701        23488   
9        50300.0                                     42217         3236   
10      253211.0                                    235574       254921   
11        5370.0                                     60996        27276   
12      166986.0                                    166236       214906   
13      106137.0                                     83267           87   
14       68684.0                                     38275            -   
15      191187.0                                    132722        35962   
16      517440.0                                    248403            -   
17      118979.0                                    487398       125884   
18       67455.0                                     23280            -   
19      564046.0          

Agora, vamos transformar os níveis das colunas em novas colunas de valores (despivotar a tabela).

In [11]:
id_col = df_consultas.columns[0]
df_consultas_ajustado = pd.melt(df_consultas, id_vars=[id_col])
df_consultas_ajustado

,"(Subprefeitura, Unnamed: 1_level_1)",variable_0,variable_1,value
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
219,Sapopemba,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,9873.0
220,Sé,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,27847.0
221,Vila Maria/Vila Guilherme,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,16263.0
222,Vila Mariana,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,3015.0


Agora, renomeamos as colunas.

In [12]:
df_consultas_ajustado.columns = ['Subprefeitura', 'Categoria', 'Subcategoria', 'Qtd_Consultas']
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
219,Sapopemba,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,9873.0
220,Sé,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,27847.0
221,Vila Maria/Vila Guilherme,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,16263.0
222,Vila Mariana,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,3015.0


Em seguida, removemos as linhas de consultas odontológicas.

In [ ]:
df_consultas_ajustado = df_consultas_ajustado[~df_consultas_ajustado['Categoria'].str.contains('Odontológica')]
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,-
190,Vila Mariana,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,139951


Ainda, substituímos os `-` por 0 na coluna de quantidade de consultas.

In [14]:
hifen = df_consultas_ajustado['Qtd_Consultas'] == '-'
df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0
df_consultas_ajustado.loc[:, 'Qtd_Consultas'] = df_consultas_ajustado.loc[:, 'Qtd_Consultas'].astype(int)
df_consultas_ajustado

C:\Users\855395\AppData\Local\Temp\ipykernel_19192\3378969537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0


,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,139951


Por último, vamos corrigir os nomes das categorias.

In [15]:
df_consultas_ajustado['Categoria'].value_counts()

Categoria
Consulta Médica\nna Atenção Básica(2)                     64
Consulta Médica\nna Atenção Especializada                 64
Consulta Médica/Atendimento\nem Urgência/Emergência(3)    64
Name: count, dtype: int64

In [ ]:
df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica na Atenção Básica' if 'Atenção Básica' in c else c)

df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica/Atendimento em Urgência/Emergência' if 'Urgência' in c else c)

df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951


## Leitos de internação e complementares

Os dados de leitos não estão disponíveis de maneira regionalizada no boletim "Saúde em dados", então terão de ser obtidos de outra fonte. Eles estão disponíveis na plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Entretanto, os links dos arquivos gerados nessa plataforma podem não ser permanentes, então utilizaremos uma versão já baixada do arquivo csv.

Obs: caso a necessidade fosse mais geral ou a atualização mais frequente, seria possível desenvolver um *scrapper* para a plataforma, mas estrapola o escopo deste trabalho.

In [17]:
data_path = path.join('dados', 'saude')
leitos_filename = 'A161721192_29_142_217.csv'
leitos_file_path = path.join(data_path, leitos_filename)

df_leito = pd.read_csv(leitos_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA,Total
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-,95.0
1,Butantã,78,131,66,57,70,8,30,440.0
2,Campo Limpo,-,-,-,-,-,88,7,95.0
3,Capela do Socorro,78,155,96,62,38,28,-,457.0
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4,387.0
5,Cidade Ademar,-,-,-,-,-,-,14,14.0
6,Cidade Tiradentes,49,54,45,38,27,10,1,224.0
7,Ermelino Matarazzo,85,58,44,40,29,-,-,256.0
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13,387.0
9,Guaianases,40,58,46,48,33,24,-,249.0


Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [18]:
df_leito = df_leito.iloc[:32]
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA,Total
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-,95.0
1,Butantã,78,131,66,57,70,8,30,440.0
2,Campo Limpo,-,-,-,-,-,88,7,95.0
3,Capela do Socorro,78,155,96,62,38,28,-,457.0
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4,387.0
5,Cidade Ademar,-,-,-,-,-,-,14,14.0
6,Cidade Tiradentes,49,54,45,38,27,10,1,224.0
7,Ermelino Matarazzo,85,58,44,40,29,-,-,256.0
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13,387.0
9,Guaianases,40,58,46,48,33,24,-,249.0


In [19]:
cols_total = df_leito.columns.str.lower().str.contains('total')
df_leito = df_leito.loc[:,~cols_total]
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-
1,Butantã,78,131,66,57,70,8,30
2,Campo Limpo,-,-,-,-,-,88,7
3,Capela do Socorro,78,155,96,62,38,28,-
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4
5,Cidade Ademar,-,-,-,-,-,-,14
6,Cidade Tiradentes,49,54,45,38,27,10,1
7,Ermelino Matarazzo,85,58,44,40,29,-,-
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13
9,Guaianases,40,58,46,48,33,24,-


Em seguida, "despivotamos" a tabela.

In [20]:
df_leito_ajustado = df_leito.melt(id_vars=['Pref Regional'],
                                    var_name='Categoria específica',
                                    value_name='Qtd leitos')
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,-
1,Butantã,1-Cirúrgico,78
2,Campo Limpo,1-Cirúrgico,-
3,Capela do Socorro,1-Cirúrgico,78
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86
...,...,...,...
219,Sapopemba,7-Hospital/DIA,6
220,Sé,7-Hospital/DIA,69
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6
222,Vila Mariana,7-Hospital/DIA,166


Por último, substituimos os `-` por 0.

In [21]:
hifen = df_leito_ajustado['Qtd leitos'].str.contains('-')
df_leito_ajustado.loc[hifen,'Qtd leitos'] = 0
df_leito_ajustado.loc[:,'Qtd leitos'] = df_leito_ajustado.loc[:,'Qtd leitos'].astype(int)
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0
1,Butantã,1-Cirúrgico,78
2,Campo Limpo,1-Cirúrgico,0
3,Capela do Socorro,1-Cirúrgico,78
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86
...,...,...,...
219,Sapopemba,7-Hospital/DIA,6
220,Sé,7-Hospital/DIA,69
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6
222,Vila Mariana,7-Hospital/DIA,166


Posteriormente, cada categoria específica dos leitos será categorizada em Leitos de internações, leitos complementares ou leitos de UTI, de acordo com as variáveis definidas.

## Mortalidade

Os dados sobre mortalidade também serão extraídos da plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Novamente, como os links dos arquivos gerados nessa plataforma podem não ser permanentes, utilizaremos uma versão já baixada do arquivo csv.

In [ ]:
data_path = path.join('dados', 'saude')
mortalidade_filename = 'A180859192_29_141_183.csv'
mortalidade_file_path = path.join(data_path, mortalidade_filename)

df_mortalidade = pd.read_csv(mortalidade_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,...,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte,Total
0,Aricanduva/Formosa/Carrão,16.0,6.0,-,38.0,6.0,788.0,3,8,22.0,...,63.0,15.0,21.0,53.0,11.0,10.0,24.0,2,507.0,3266.0
1,Butantã,14.0,4.0,-,30.0,14.0,696.0,5,13,34.0,...,86.0,30.0,40.0,44.0,26.0,15.0,34.0,4,646.0,3615.0
2,Campo Limpo,13.0,10.0,-,43.0,20.0,1005.0,5,14,42.0,...,81.0,62.0,49.0,77.0,25.0,55.0,46.0,5,673.0,4409.0
3,Capela do Socorro,22.0,19.0,-,52.0,21.0,1070.0,9,11,44.0,...,109.0,51.0,61.0,71.0,17.0,46.0,31.0,4,793.0,4821.0
4,Casa Verde/Cachoeirinha,12.0,16.0,-,39.0,19.0,794.0,7,14,36.0,...,72.0,18.0,39.0,48.0,14.0,26.0,16.0,4,501.0,3216.0
5,Cidade Ademar,10.0,19.0,-,27.0,15.0,764.0,5,26,33.0,...,89.0,31.0,46.0,51.0,20.0,33.0,27.0,3,519.0,3341.0
6,Cidade Tiradentes,2.0,11.0,-,21.0,11.0,415.0,3,9,11.0,...,42.0,18.0,9.0,35.0,8.0,16.0,10.0,11,202.0,1649.0
7,Ermelino Matarazzo,11.0,8.0,-,19.0,10.0,488.0,2,8,22.0,...,56.0,27.0,18.0,29.0,10.0,13.0,12.0,7,334.0,2109.0
8,Freguesia/Brasilândia,3.0,10.0,-,41.0,13.0,985.0,9,16,27.0,...,134.0,32.0,43.0,67.0,19.0,33.0,27.0,4,599.0,3982.0
9,Guaianases,10.0,14.0,1,38.0,16.0,510.0,3,11,13.0,...,64.0,21.0,22.0,51.0,17.0,22.0,19.0,5,336.0,2263.0


Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [ ]:
df_mortalidade = df_mortalidade.iloc[:32]
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,...,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte,Total
0,Aricanduva/Formosa/Carrão,16.0,6.0,-,38.0,6.0,788.0,3,8,22.0,...,63.0,15.0,21.0,53.0,11.0,10.0,24.0,2,507.0,3266.0
1,Butantã,14.0,4.0,-,30.0,14.0,696.0,5,13,34.0,...,86.0,30.0,40.0,44.0,26.0,15.0,34.0,4,646.0,3615.0
2,Campo Limpo,13.0,10.0,-,43.0,20.0,1005.0,5,14,42.0,...,81.0,62.0,49.0,77.0,25.0,55.0,46.0,5,673.0,4409.0
3,Capela do Socorro,22.0,19.0,-,52.0,21.0,1070.0,9,11,44.0,...,109.0,51.0,61.0,71.0,17.0,46.0,31.0,4,793.0,4821.0
4,Casa Verde/Cachoeirinha,12.0,16.0,-,39.0,19.0,794.0,7,14,36.0,...,72.0,18.0,39.0,48.0,14.0,26.0,16.0,4,501.0,3216.0
5,Cidade Ademar,10.0,19.0,-,27.0,15.0,764.0,5,26,33.0,...,89.0,31.0,46.0,51.0,20.0,33.0,27.0,3,519.0,3341.0
6,Cidade Tiradentes,2.0,11.0,-,21.0,11.0,415.0,3,9,11.0,...,42.0,18.0,9.0,35.0,8.0,16.0,10.0,11,202.0,1649.0
7,Ermelino Matarazzo,11.0,8.0,-,19.0,10.0,488.0,2,8,22.0,...,56.0,27.0,18.0,29.0,10.0,13.0,12.0,7,334.0,2109.0
8,Freguesia/Brasilândia,3.0,10.0,-,41.0,13.0,985.0,9,16,27.0,...,134.0,32.0,43.0,67.0,19.0,33.0,27.0,4,599.0,3982.0
9,Guaianases,10.0,14.0,1,38.0,16.0,510.0,3,11,13.0,...,64.0,21.0,22.0,51.0,17.0,22.0,19.0,5,336.0,2263.0


In [24]:
cols_total = df_mortalidade.columns.str.lower().str.contains('total')
df_mortalidade = df_mortalidade.loc[:,~cols_total]
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,...,Demais anomalias congênitas,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte
0,Aricanduva/Formosa/Carrão,16.0,6.0,-,38.0,6.0,788.0,3,8,22.0,...,5,63.0,15.0,21.0,53.0,11.0,10.0,24.0,2,507.0
1,Butantã,14.0,4.0,-,30.0,14.0,696.0,5,13,34.0,...,9,86.0,30.0,40.0,44.0,26.0,15.0,34.0,4,646.0
2,Campo Limpo,13.0,10.0,-,43.0,20.0,1005.0,5,14,42.0,...,8,81.0,62.0,49.0,77.0,25.0,55.0,46.0,5,673.0
3,Capela do Socorro,22.0,19.0,-,52.0,21.0,1070.0,9,11,44.0,...,13,109.0,51.0,61.0,71.0,17.0,46.0,31.0,4,793.0
4,Casa Verde/Cachoeirinha,12.0,16.0,-,39.0,19.0,794.0,7,14,36.0,...,2,72.0,18.0,39.0,48.0,14.0,26.0,16.0,4,501.0
5,Cidade Ademar,10.0,19.0,-,27.0,15.0,764.0,5,26,33.0,...,8,89.0,31.0,46.0,51.0,20.0,33.0,27.0,3,519.0
6,Cidade Tiradentes,2.0,11.0,-,21.0,11.0,415.0,3,9,11.0,...,3,42.0,18.0,9.0,35.0,8.0,16.0,10.0,11,202.0
7,Ermelino Matarazzo,11.0,8.0,-,19.0,10.0,488.0,2,8,22.0,...,5,56.0,27.0,18.0,29.0,10.0,13.0,12.0,7,334.0
8,Freguesia/Brasilândia,3.0,10.0,-,41.0,13.0,985.0,9,16,27.0,...,7,134.0,32.0,43.0,67.0,19.0,33.0,27.0,4,599.0
9,Guaianases,10.0,14.0,1,38.0,16.0,510.0,3,11,13.0,...,6,64.0,21.0,22.0,51.0,17.0,22.0,19.0,5,336.0


Em seguida, "despivotamos" a tabela.

In [25]:
df_mortalidade_ajustado = df_mortalidade.melt(id_vars=['Subprefeitura residência'],
                                    var_name='Causa específica',
                                    value_name='Qtd óbitos')
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16.0
1,Butantã,Doencas infec intest,14.0
2,Campo Limpo,Doencas infec intest,13.0
3,Capela do Socorro,Doencas infec intest,22.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,12.0
...,...,...,...
2139,Sapopemba,Demais causas de morte,429.0
2140,Sé,Demais causas de morte,733.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523.0
2142,Vila Mariana,Demais causas de morte,682.0


Por último, substituimos os `-` por 0.

In [ ]:
hifen = df_mortalidade_ajustado['Qtd óbitos'].astype(str).str.contains('-')
df_mortalidade_ajustado.loc[hifen,'Qtd óbitos'] = 0
df_mortalidade_ajustado.loc[:,'Qtd óbitos'] = df_mortalidade_ajustado.loc[:,'Qtd óbitos'].astype(int)
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16
1,Butantã,Doencas infec intest,14
2,Campo Limpo,Doencas infec intest,13
3,Capela do Socorro,Doencas infec intest,22
4,Casa Verde/Cachoeirinha,Doencas infec intest,12
...,...,...,...
2139,Sapopemba,Demais causas de morte,429
2140,Sé,Demais causas de morte,733
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523
2142,Vila Mariana,Demais causas de morte,682


Posteriormente, cada causa específica dos óbitos será categorizado em Evitável, Tratável ou Evitável/Tratável, de acordo com uma tabela auxiliar definida pelo GT.

## Despesas

Para que seja possível relacionar as despesas com os outros indicadores, podemos usar os dados de regionalização do orçamento, disponíveis de acordo com o valor de detalhamento da ação.

Esses dados ficam disponíveis na [página de prestação de contas públicas da Secretaria Municipal da Fazenda](https://orcamento.sf.prefeitura.sp.gov.br/orcamento/execucao.php), na Base de Dados da Regionalização da Execução Orçamentária.

In [27]:
despesas_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2023/basedadosDA_1223.csv'
df_despesa = pd.read_csv(despesas_url,
                         encoding='latin1',
                         sep=';',
                         decimal=',')

df_despesa

C:\Users\855395\AppData\Local\Temp\ipykernel_19192\4118471245.py:2: DtypeWarning: Columns (27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_despesa = pd.read_csv(despesas_url,


,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,2305,2023,6655,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,4850.00,03/06/2025
1,4,2389,2023,39,2024,2024-01-18 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,126348.08,03/06/2025
2,4,2389,2023,6385,2023,2023-11-10 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,150000.00,03/06/2025
3,4,2389,2023,6654,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,223651.92,03/06/2025
4,4,3280,2023,120,2024,2024-01-19 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,19139.06,03/06/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476471,1,110546,2023,30319,2024,2024-02-06 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,1,751,253,0,NaN,NaN,NaN,NaN,NaN,03/06/2025
476472,1,110546,2023,30335,2024,2024-02-06 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,1,751,253,0,NaN,NaN,NaN,NaN,NaN,03/06/2025
476473,1,110547,2023,30339,2024,2024-02-06 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,1,751,253,1,NaN,NaN,NaN,NaN,NaN,03/06/2025
476474,1,110548,2023,30341,2024,2024-02-06 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,...,2,751,253,1,NaN,NaN,NaN,NaN,NaN,03/06/2025


Primeiro, vamos filtrar os dados para manter apenas a função Saúde do orçamento.

In [28]:
df_despesa = df_despesa[df_despesa['DESCRIÇÃO_FUNÇÃO']=='Saúde']
df_despesa

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,2305,2023,6655,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,4850.00,03/06/2025
1,4,2389,2023,39,2024,2024-01-18 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,126348.08,03/06/2025
2,4,2389,2023,6385,2023,2023-11-10 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,150000.00,03/06/2025
3,4,2389,2023,6654,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,223651.92,03/06/2025
4,4,3280,2023,120,2024,2024-01-19 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,19139.06,03/06/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434572,1,107178,2023,18126,2024,2024-01-24 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,28,...,1,500,9001,1,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,78.78,03/06/2025
434573,1,107178,2023,288651,2023,2023-11-22 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,28,...,1,500,9001,1,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,66.84,03/06/2025
434574,1,107178,2023,291115,2023,2023-11-23 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,28,...,1,500,9001,1,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,84.63,03/06/2025
434575,1,107178,2023,323764,2023,2023-12-21 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,28,...,1,500,9001,1,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,86.17,03/06/2025


In [29]:
group_cols = ['CÓDIGO_SUBFUNÇÃO',
        'DESCRIÇÃO_SUBFUNÇÃO',
        'CÓDIGO_PROGRAMA',
        'DESCRIÇÃO_PROGRAMA',
        'REGIÃO',
        'SUBPREFEITURA',
        'DISTRITO',
        'TIPO_REGIONALIZAÇÃO']

metric_cols = ['VALOR_DETALHAMENTO_AÇÃO']

df_despesa_agrupado = df_despesa.groupby(group_cols).sum().reset_index().loc[:, group_cols + metric_cols]
df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09
...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05


Vamos ver quais valores aparecem como subfunção.

In [30]:
df_despesa_agrupado['DESCRIÇÃO_PROGRAMA'].value_counts()

DESCRIÇÃO_PROGRAMA
Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância      88
Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência    38
Ações e Serviços da Saúde Animal                                              10
Suporte Administrativo                                                         4
Modernização Tecnológica, Desburocratização e Inovação do Serviço Público      3
Benefícios e Previdência de Funcionários                                       1
Participação, Transparência e Controle Social da Administração Pública         1
Name: count, dtype: int64

Para os indicadores de "Despesas liquidadas em atenção básica e vigilância" e "Despesas liquidadas em atenção especializada e Suporte profilático terapêutico", parece que os programas serão o suficiente, com alguns não se encaixando em nenhuma das categorias.

Baixados todos os dados, vamos começar as transformações para adequar as categorias dos dados às necessidades dos indicadores.

# Transformação e padronização

Primeiro, vamos criar as variáveis que representarão os níveis de atenção padronizados.

In [31]:
NIVEL_COL_NAME = 'Nível de atenção padronizado'
NIVEL_AB_VIGILANCIA = 'Atenção Básica e Vigilância'
NIVEL_ESP = 'Atenção Especializada e Suporte profilático terapêutico'
NIVEL_OUTROS = 'Outros níveis de atenção'

Nas próximas seções, vamos também padronizar os nomes das Subprefeituras, com uma função já disponível no módulo `core.utils.cleaning`.

## Consultas médicas

In [32]:
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951


In [ ]:
df_consultas_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (
    clean_subprefeitura(df_consultas_ajustado.loc[:, 'Subprefeitura'])
)

df_consultas_ajustado

C:\Users\855395\AppData\Local\Temp\ipykernel_19192\1073107716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consultas_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (


,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas,SUBPREFEITURA_QLIK
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894,ARICANDUVA/FORMOSA/CARRAO
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917,BUTANTA
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251,CAMPO LIMPO
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063,CAPELA DO SOCORRO
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363,CASA VERDE/CACHOEIRINHA
...,...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822,SAPOPEMBA
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857,SE
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0,VILA MARIA/VILA GUILHERME
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951,VILA MARIANA


## Leitos de internação e complementares

Primeiro, vamos organizar os leitos nas categorias: Leitos de internações; Leitos complementares.

A regra específicada pelo GT foi a seguinte:

 - Leitos de internações são todas as categorias de leito, exceto as categorias `3-Complementar` e `7-Hospital/DIA`;

 - Leitos complementares são os da categoria `3-Complementar`;

In [34]:
df_leito_ajustado['Categoria específica'].value_counts()

Categoria específica
1-Cirúrgico                32
2-Clínico                  32
3-Complementar             32
4-Obstétrico               32
5-Pediátrico               32
6-Outras Especialidades    32
7-Hospital/DIA             32
Name: count, dtype: int64

In [35]:
leito_map = {
    '1-Cirúrgico': 'Leitos de internação',
    '2-Clínico': 'Leitos de internação',
    '3-Complementar': 'Leitos complementares',
    '4-Obstétrico': 'Leitos de internação',
    '5-Pediátrico': 'Leitos de internação',
    '6-Outras Especialidades': 'Leitos de internação',
    '7-Hospital/DIA': None,
}

df_leito_ajustado.loc[:, 'Categoria geral'] = df_leito_ajustado['Categoria específica'].map(leito_map)
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos,Categoria geral
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0,Leitos de internação
1,Butantã,1-Cirúrgico,78,Leitos de internação
2,Campo Limpo,1-Cirúrgico,0,Leitos de internação
3,Capela do Socorro,1-Cirúrgico,78,Leitos de internação
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86,Leitos de internação
...,...,...,...,...
219,Sapopemba,7-Hospital/DIA,6,None
220,Sé,7-Hospital/DIA,69,None
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6,None
222,Vila Mariana,7-Hospital/DIA,166,None


Agora, adicionamos também a classificação geral por nível de atenção (básica ou especializada). Para leitos, todos são utilizados na Atenção especializada, então é só adicionarmos uma coluna com o valor como padrão.

In [36]:
df_leito_ajustado.loc[:, NIVEL_COL_NAME] = NIVEL_ESP
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos,Categoria geral,Nível de atenção padronizado
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0,Leitos de internação,Atenção Especializada e Suporte profilático te...
1,Butantã,1-Cirúrgico,78,Leitos de internação,Atenção Especializada e Suporte profilático te...
2,Campo Limpo,1-Cirúrgico,0,Leitos de internação,Atenção Especializada e Suporte profilático te...
3,Capela do Socorro,1-Cirúrgico,78,Leitos de internação,Atenção Especializada e Suporte profilático te...
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86,Leitos de internação,Atenção Especializada e Suporte profilático te...
...,...,...,...,...,...
219,Sapopemba,7-Hospital/DIA,6,None,Atenção Especializada e Suporte profilático te...
220,Sé,7-Hospital/DIA,69,None,Atenção Especializada e Suporte profilático te...
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6,None,Atenção Especializada e Suporte profilático te...
222,Vila Mariana,7-Hospital/DIA,166,None,Atenção Especializada e Suporte profilático te...


In [37]:
group_cols = ['Pref Regional',
              'Categoria geral',
              NIVEL_COL_NAME]

df_leitos_final = (
    df_leito_ajustado[group_cols + ['Qtd leitos']]
    .groupby(group_cols)
    .sum()
    .reset_index()
)

df_leitos_final

,Pref Regional,Categoria geral,Nível de atenção padronizado,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55


In [38]:
df_leitos_final = df_leitos_final.rename(columns={'Pref Regional': 'Subprefeitura'})
df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55


In [39]:
df_leitos_final.loc[:, 'SUBPREFEITURA_QLIK'] = (
    clean_subprefeitura(df_leitos_final.loc[:, 'Subprefeitura'])
)

df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos,SUBPREFEITURA_QLIK
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,ARICANDUVA/ FORMOSA/ CARRAO
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95,ARICANDUVA/ FORMOSA/ CARRAO
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66,BUTANTA
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344,BUTANTA
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,CAMPO LIMPO
...,...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144,VILA MARIA/ VILA GUILHERME
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324,VILA MARIANA
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491,VILA MARIANA
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55,VILA PRUDENTE


## Despesas

Para manter mais flexibilidade, vamos manter todas as colunas presentes na tabela de despesas, mesmo que usemos apenas a coluna de programa. Então, o que vamos fazer aqui é criar a nova coluna de nível de atenção padronizado com base nos programas.

In [40]:
df_despesa_agrupado['DESCRIÇÃO_PROGRAMA'].value_counts()

DESCRIÇÃO_PROGRAMA
Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância      88
Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência    38
Ações e Serviços da Saúde Animal                                              10
Suporte Administrativo                                                         4
Modernização Tecnológica, Desburocratização e Inovação do Serviço Público      3
Benefícios e Previdência de Funcionários                                       1
Participação, Transparência e Controle Social da Administração Pública         1
Name: count, dtype: int64

In [41]:
map_despesa = {
    'Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância': NIVEL_AB_VIGILANCIA,
    'Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência': NIVEL_ESP
}

df_despesa_agrupado.loc[:, NIVEL_COL_NAME] = df_despesa_agrupado.loc[:, 'DESCRIÇÃO_PROGRAMA'].map(map_despesa)
df_despesa_agrupado.loc[:, NIVEL_COL_NAME] = df_despesa_agrupado.loc[:, NIVEL_COL_NAME].fillna(NIVEL_OUTROS)
df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,Nível de atenção padronizado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07,Atenção Básica e Vigilância
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06,Outros níveis de atenção
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06,Outros níveis de atenção
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08,Outros níveis de atenção
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09,Outros níveis de atenção
...,...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05,Outros níveis de atenção
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06,Outros níveis de atenção
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06,Outros níveis de atenção
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05,Outros níveis de atenção


Vamos também criar uma nova coluna com o nome da subprefeitura, de acordo com o padrão existente no arquivo com os polígonos do Qlik.

In [42]:
subpref_filter = (
    df_despesa_agrupado.loc[:, 'SUBPREFEITURA']
    .str.startswith('Subprefeitura ')
)

df_despesa_agrupado.loc[subpref_filter, 'SUBPREFEITURA_QLIK'] = (
    df_despesa_agrupado
    .loc[subpref_filter, 'SUBPREFEITURA']
    .str.removeprefix('Subprefeitura ')
    .str.upper()
)

df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,Nível de atenção padronizado,SUBPREFEITURA_QLIK
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07,Atenção Básica e Vigilância,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06,Outros níveis de atenção,SÉ
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06,Outros níveis de atenção,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08,Outros níveis de atenção,SÉ
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09,Outros níveis de atenção,NaN
...,...,...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05,Outros níveis de atenção,SANTANA/TUCURUVI
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06,Outros níveis de atenção,BUTANTÃ
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06,Outros níveis de atenção,PINHEIROS
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05,Outros níveis de atenção,SANTO AMARO


In [43]:
df_despesa_agrupado.loc[subpref_filter, 'SUBPREFEITURA_QLIK'].apply(unidecode).str.replace("'", ' ').value_counts()

SUBPREFEITURA_QLIK
SE                           9
BUTANTA                      6
CASA VERDE/CACHOEIRINHA      5
ARICANDUVA/FORMOSA/CARRAO    5
SANTANA/TUCURUVI             5
SANTO AMARO                  5
MOOCA                        4
ERMELINO MATARAZZO           4
JACANA/TREMEMBE              4
PIRITUBA/JARAGUA             4
PINHEIROS                    4
CAMPO LIMPO                  4
VILA MARIA/VILA GUILHERME    4
FREGUESIA/BRASILANDIA        4
SAPOPEMBA                    4
ITAQUERA                     4
LAPA                         3
CAPELA DO SOCORRO            3
DE GUAIANASES                3
DE VILA PRUDENTE             3
PENHA                        3
CIDADE TIRADENTES            3
SAO MIGUEL PAULISTA          3
PERUS/ANHANGUERA             3
JABAQUARA                    3
M BOI MIRIM                  3
IPIRANGA                     3
CIDADE ADEMAR                3
PARELHEIROS                  3
VILA MARIANA                 3
ITAIM PAULISTA               2
SAO MATEUS          

In [44]:
df_despesa_agrupado['SUBPREFEITURA_QLIK'] = clean_subprefeitura(df_despesa_agrupado['SUBPREFEITURA_QLIK'])

df_despesa_agrupado

AttributeError: 'float' object has no attribute 'encode'

In [42]:
df_despesa_agrupado['SUBPREFEITURA_QLIK'].value_counts()

SUBPREFEITURA_QLIK
SE                           9
BUTANTA                      6
CASA VERDE/CACHOEIRINHA      5
ARICANDUVA/FORMOSA/CARRAO    5
SANTANA/TUCURUVI             5
SANTO AMARO                  5
MOOCA                        4
ERMELINO MATARAZZO           4
JACANA/TREMEMBE              4
PIRITUBA/JARAGUA             4
PINHEIROS                    4
CAMPO LIMPO                  4
VILA MARIA/VILA GUILHERME    4
FREGUESIA/BRASILANDIA        4
SAPOPEMBA                    4
ITAQUERA                     4
LAPA                         3
CAPELA DO SOCORRO            3
DE GUAIANASES                3
DE VILA PRUDENTE             3
PENHA                        3
CIDADE TIRADENTES            3
SAO MIGUEL PAULISTA          3
PERUS/ANHANGUERA             3
JABAQUARA                    3
M BOI MIRIM                  3
IPIRANGA                     3
CIDADE ADEMAR                3
PARELHEIROS                  3
VILA MARIANA                 3
ITAIM PAULISTA               2
SAO MATEUS          